In [ ]:
#accessing Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#libraries
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de Confusão',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Classificação Verdadeira')
    plt.xlabel('Classificação Prevista')

def number_of_labels(number_labels):
  if number_labels == 4:

    #keeping 4 labels
    #(changing labels range)
    test_labels[test_labels == 1] = 0
    test_labels[test_labels == 2] = 1
    test_labels[test_labels == 3] = 2
    test_labels[test_labels == 4] = 3

  elif number_labels == 3:

    #changing from 4 to 3 labels
    #(merging partially deforested and partially preserved)
    test_labels[test_labels == 3] = 2
    test_labels[test_labels == 1] = 0
    test_labels[test_labels == 2] = 1
    test_labels[test_labels == 4] = 2

  elif number_labels == 2:

    #changing from 4 to 2 labels
    #(merging partially deforested, partially preserved
    #and totally deforested)
    test_labels[test_labels == 3] = 1
    test_labels[test_labels == 2] = 1
    test_labels[test_labels == 1] = 0
    test_labels[test_labels == 4] = 1

In [ ]:
#loading testing dataset

test_images = np.load("/content/drive/MyDrive/TCC2/npy_files/"
                      + "test_images.npy")
test_labels = np.load("/content/drive/MyDrive/TCC2/npy_files/"
                      + "test_labels.npy")

In [ ]:
#choosing the number of labels

number_labels = int(input("Number of labels? "))
number_epochs = int(input("Number of epochs? "))

number_of_labels(number_labels)

In [ ]:
#loading the model

model = load_model('/content/drive/MyDrive/TCC2/models/cnn_'
                    + str(number_labels) + '_Labels_' + str(number_epochs)
                    + '_epochs.h5')
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
#making prediction with test dataset

predictions = model.predict(x=test_images, steps=len(test_images),
                            verbose=1)
predictions = np.round(predictions)

In [ ]:
#plotting confusion matrix

cm = confusion_matrix(y_true=test_labels,
                      y_pred=np.argmax(predictions, axis=-1))
if number_labels == 4:
  cm_plot_labels = ['Totalmente Desmatado',
                    'Parcialmente Desmatado',
                    'Parcialmente Preservado',
                    'Totalmente Preservado']
elif number_labels == 3:
  cm_plot_labels = ['Totalmente Desmatado',
                    'Parcialmente Desamatado',
                    'Totalmente Preservado']
elif number_labels == 2:
  cm_plot_labels = ['Desmatado', 'Preservado']

plot_confusion_matrix(cm=cm, classes=cm_plot_labels)